In [ ]:
from steel_design import SteelDesign

class BoltedConnectionDesign(SteelDesign):
    """Class for bolted connection design."""
    def __init__(self, steel_grade, yield_strength, bolt_grade, bolt_size, num_bolts):
        super().__init__(steel_grade, yield_strength)
        self.bolt_grade = bolt_grade
        self.bolt_size = bolt_size
        self.num_bolts = num_bolts

    def calculate_bolt_shear_capacity(self):
        """Calculate shear capacity of bolts (BS 5950, Clause 6.3)."""
        return self.calculate_bolt_shear_capacity(self.bolt_grade, self.bolt_size, self.num_bolts)

def main():
    # Example 4.18 parameters
    connection = BoltedConnectionDesign(
        steel_grade="S275",
        yield_strength=275,
        bolt_grade="HSFG ≤ M24",
        bolt_size="M22",
        num_bolts=4
    )
    
    applied_load = 500  # kN
    capacity = connection.calculate_bolt_shear_capacity()
    
    print(f"Example 4.18: Applied Load = {applied_load:.1f} kN, Bolt Shear Capacity = {capacity:.1f} kN")
    print(f"Check: {'OK' if capacity >= applied_load else 'Not OK'}")

if __name__ == "__main__":
    main()

In [ ]:
from steel_design import SteelDesign

class BoltedConnectionDesign(SteelDesign):
    """Class for bolted connection design."""
    def __init__(self, steel_grade, yield_strength, bolt_grade, bolt_size, num_bolts):
        super().__init__(steel_grade, yield_strength)
        self.bolt_grade = bolt_grade
        self.bolt_size = bolt_size
        self.num_bolts = num_bolts

    def calculate_bolt_tension_capacity(self):
        """Calculate tension capacity of bolts (BS 5950, Clause 6.3)."""
        pt = get_bolt_shear_strength(self.bolt_grade) * 0.8  # Simplified tension strength
        bolt_area = get_bolt_area(self.bolt_size)
        return self.num_bolts * pt * bolt_area / 1000  # kN

def main():
    # Example 4.17 parameters
    connection = BoltedConnectionDesign(
        steel_grade="S275",
        yield_strength=275,
        bolt_grade="8.8",
        bolt_size="M20",
        num_bolts=4
    )
    
    applied_load = 200  # kN
    capacity = connection.calculate_bolt_tension_capacity()
    
    print(f"Example 4.17: Applied Load = {applied_load:.1f} kN, Bolt Tension Capacity = {capacity:.1f} kN")
    print(f"Check: {'OK' if capacity >= applied_load else 'Not OK'}")

if __name__ == "__main__":
    main()

In [ ]:
from steel_design import SteelDesign

class WeldedConnectionDesign(SteelDesign):
    """Class for welded connection design."""
    def __init__(self, steel_grade, yield_strength, weld_throat, weld_length):
        super().__init__(steel_grade, yield_strength)
        self.weld_throat = weld_throat  # mm
        self.weld_length = weld_length  # mm
        self.pw = 220  # Weld strength (N/mm², BS 5950, Table 37, for E43 electrodes)

    def calculate_weld_capacity(self):
        """Calculate fillet weld capacity (BS 5950, Clause 6.8)."""
        return self.pw * self.weld_throat * self.weld_length / 1000  # kN

def main():
    # Example 4.16 parameters
    connection = WeldedConnectionDesign(
        steel_grade="S275",
        yield_strength=275,
        weld_throat=6,  # mm
        weld_length=200  # mm (assumed)
    )
    
    applied_load = 300  # kN
    capacity = connection.calculate_weld_capacity()
    
    print(f"Example 4.16: Applied Load = {applied_load:.1f} kN, Weld Capacity = {capacity:.1f} kN")
    print(f"Check: {'OK' if capacity >= applied_load else 'Not OK'}")

if __name__ == "__main__":
    main()

In [ ]:
# example_4_15.py
from steel_design import SteelDesign
from dataset import get_load_factor

class TensionDesign(SteelDesign):
    """Class for tension member design."""
    def __init__(self, steel_grade, yield_strength, gross_area):
        super().__init__(steel_grade, yield_strength)
        self.Ag = gross_area  # mm²

    def calculate_tension_capacity(self):
        """Calculate tension capacity (BS 5950, Clause 4.6)."""
        return self.yield_strength * self.Ag / 1000  # kN

def main():
    # Example 4.15 parameters
    tension_member = TensionDesign(
        steel_grade="S275",
        yield_strength=275,
        gross_area=5000  # mm²
    )
    
    applied_load = tension_member.apply_load_factor(400, "Imposed Vertical Load")
    capacity = tension_member.calculate_tension_capacity()
    
    print(f"Example 4.15: Factored Load = {applied_load:.1f} kN, Tension Capacity = {capacity:.1f} kN")
    print(f"Check: {'OK' if capacity >= applied_load else 'Not OK'}")

if __name__ == "__main__":
    main()

In [ ]:
# example_4_13.py
from steel_design import SteelDesign

class BeamDesign(SteelDesign):
    """Class for beam design calculations."""
    def __init__(self, steel_grade, yield_strength, plastic_modulus, depth, web_thickness):
        super().__init__(steel_grade, yield_strength)
        self.Sx = plastic_modulus  # mm³
        self.depth = depth  # mm
        self.t = web_thickness  # mm

    def calculate_moment_capacity(self):
        """Calculate moment capacity (BS 5950, Clause 4.2.5)."""
        return self.yield_strength * self.Sx / 1e6  # kNm

    def calculate_shear_capacity(self):
        """Calculate shear capacity (BS 5950, Clause 4.2.3)."""
        Av = self.depth * self.t
        return 0.6 * self.yield_strength * Av / 1000  # kN

    def check_combined_loading(self, applied_moment, applied_shear):
        """Check combined bending and shear (BS 5950, Clause 4.2.6)."""
        Mc = self.calculate_moment_capacity()
        Pv = self.calculate_shear_capacity()
        return applied_moment <= Mc and applied_shear <= Pv

def main():
    # Example 4.13 parameters
    beam = BeamDesign(
        steel_grade="S275",
        yield_strength=275,
        plastic_modulus=2300e3,  # mm³
        depth=400,  # mm
        web_thickness=10  # mm
    )
    
    applied_moment = 600  # kNm
    applied_shear = 300  # kN
    is_adequate = beam.check_combined_loading(applied_moment, applied_shear)
    
    print(f"Example 4.13: Applied Moment = {applied_moment:.1f} kNm, Moment Capacity = {beam.calculate_moment_capacity():.1f} kNm")
    print(f"Applied Shear = {applied_shear:.1f} kN, Shear Capacity = {beam.calculate_shear_capacity():.1f} kN")
    print(f"Combined Loading Check: {'OK' if is_adequate else 'Not OK'}")

if __name__ == "__main__":
    main()

In [ ]:
# example_4_12.py
from steel_design import SteelDesign
from dataset import get_bending_strength, get_effective_length_factor

class BeamDesign(SteelDesign):
    """Class for beam design calculations."""
    def __init__(self, steel_grade, yield_strength, span, plastic_modulus, slenderness, d_t_ratio):
        super().__init__(steel_grade, yield_strength)
        self.span = span  # mm
        self.Sx = plastic_modulus  # mm³
        self.slenderness = slenderness
        self.d_t_ratio = d_t_ratio

    def calculate_buckling_resistance_moment(self, restraint_condition, loading_condition):
        """Calculate buckling resistance moment (BS 5950, Clause 4.3.6)."""
        pb = get_bending_strength(self.steel_grade, self.slenderness, self.d_t_ratio)
        Le = self.calculate_effective_length(self.span, restraint_condition, loading_condition)
        # Assume slenderness adjusted for Le (simplified)
        return pb * self.Sx / 1e6  # kNm

def main():
    # Example 4.12 parameters
    beam = BeamDesign(
        steel_grade="S275",
        yield_strength=265,
        span=7000,  # mm
        plastic_modulus=2300e3,  # mm³
        slenderness=90,
        d_t_ratio=16.8
    )
    
    buckling_moment = beam.calculate_buckling_resistance_moment(
        restraint_condition="Compression Flange Laterally Restrained",
        loading_condition="Normal"
    )
    print(f"Example 4.12: Buckling Resistance Moment = {buckling_moment:.1f} kNm")

if __name__ == "__main__":
    main()

In [ ]:
# example_4_11.py
from steel_design import SteelDesign
from dataset import get_bending_strength, get_buckling_parameters

class BeamDesign(SteelDesign):
    """Class for beam design calculations."""
    def __init__(self, steel_grade, yield_strength, section, plastic_modulus, slenderness, d_t_ratio):
        super().__init__(steel_grade, yield_strength)
        self.section = section
        self.Sx = plastic_modulus  # mm³
        self.slenderness = slenderness
        self.d_t_ratio = d_t_ratio

    def calculate_buckling_resistance_moment(self):
        """Calculate buckling resistance moment (BS 5950, Clause 4.3.6)."""
        pb = get_bending_strength(self.steel_grade, self.slenderness, self.d_t_ratio)
        params = self.get_buckling_parameters(self.section)
        u = params["u"]
        # Adjust pb for buckling parameters if needed (simplified here)
        return pb * self.Sx / 1e6  # kNm

def main():
    # Example 4.11 parameters
    beam = BeamDesign(
        steel_grade="S275",
        yield_strength=265,
        section="305x305x150 UC",
        plastic_modulus=2300e3,  # mm³ (assumed)
        slenderness=90,
        d_t_ratio=16.8
    )
    
    buckling_moment = beam.calculate_buckling_resistance_moment()
    print(f"Example 4.11: Buckling Resistance Moment = {buckling_moment:.1f} kNm")

if __name__ == "__main__":
    main()

In [ ]:
# steel_design.py
from dataset import get_load_factor, get_deflection_limit, get_bending_strength, get_compressive_strength, get_effective_length_factor, get_buckling_parameters

class SteelDesign:
    """Base class for structural steel design to BS 5950."""
    def __init__(self, steel_grade="S275", yield_strength=275):
        self.steel_grade = steel_grade
        self.yield_strength = yield_strength  # N/mm²
        self.E = 205000  # Elastic modulus (N/mm²)

    def apply_load_factor(self, load, load_type):
        """Apply load factor from Table 4.2."""
        factor = get_load_factor(load_type)
        return load * factor

    def calculate_effective_length(self, actual_length, restraint_condition, loading_condition):
        """Calculate effective length using Table 4.6."""
        factor = get_effective_length_factor(restraint_condition, loading_condition)
        return actual_length * factor

    def get_buckling_parameters(self, section):
        """Retrieve buckling parameters (u, x) from Tableelectron

System: Table 4.15."""
        return get_buckling_parameters(section)

In [ ]:
# dataset.py
import pandas as pd

# Table 4.2: Partial factors for loads (BS 5950, Table 2)
table_4_2 = {
    "Dead Load": 1.4,
    "Dead Load with Wind and Imposed Load": 1.4,
    "Imposed Vertical Load": 1.6,
}

def get_load_factor(load_type):
    """Retrieve load factor from Table 4.2."""
    return table_4_2.get(load_type, 1.4)

# Table 4.5: Suggested vertical deflection limits on beams (BS 5950, Table 8)
table_4_5 = {
    "Cantilevers": "Length/180",
    "Beams with Plaster or Brittle Finish": "Span/360",
    "Other Beams": "Span/200"
}

def get_deflection_limit(beam_type):
    """Retrieve deflection limit from Table 4.5."""
    return table_4_5.get(beam_type, "Span/200")

# Table 4.6: Effective length (Le) for beams (BS 5950, Table 13)
table_4_6 = {
    "Compression Flange Fully Restrained": {
        "Normal": 1.0,
        "Destabilizing": 1.2
    },
    "Compression Flange Laterally Restrained": {
        "Normal": 0.85,
        "Destabilizing": 1.0
    }
}

def get_effective_length_factor(restraint_condition, loading_condition):
    """Retrieve effective length factor from Table 4.6."""
    return table_4_6.get(restraint_condition, {}).get(loading_condition, 1.0)

# Table 4.7: Bending strength (pb) based on slenderness and D/T ratio
table_4_7 = {
    "S275": {
        (90, 16.8): 196,  # (slenderness λ, D/T) -> pb (N/mm²)
        (25.7, 16.8): 265,
    },
    "S355": {
        # Placeholder
    }
}

def get_bending_strength(steel_grade, slenderness, d_t_ratio):
    """Retrieve bending strength (pb) from Table 4.7."""
    return table_4_7.get(steel_grade, {}).get((slenderness, d_t_ratio), 275)

# Table 4.14: Compressive strength (pc) for strut curve c (BS 5950, Table 24(c))
table_4_14 = {
    "S275": {
        45: 222,  # slenderness (λ) -> pc (N/mm²)
        25.7: 257,
        41.2: 257,
    },
    "S355": {
        # Placeholder
    }
}

def get_compressive_strength(steel_grade, slenderness):
    """Retrieve compressive strength (pc) from Table 4.14."""
    return table_4_14.get(steel_grade, {}).get(slenderness, 275)

# Table 4.15: Buckling parameter (u) and torsional index (x) for angles (BS 5950, Annex I)
table_4_15 = {
    "305x305x150 UC": {
        "u": 0.87,
        "x": 35.0
    },
    # Add more sections as needed
}

def get_buckling_parameters(section):
    """Retrieve buckling parameter (u) and torsional index (x) from Table 4.15."""
    return table_4_15.get(section, {"u": 0.85, "x": 30.0})

In [ ]:
# example_4_10.py
from steel_design import SteelDesign
from dataset import get_load_factor, get_compressive_strength

class ColumnDesign(SteelDesign):
    """Class for column design calculations."""
    def __init__(self, steel_grade, yield_strength, length, area, radius_gyration):
        super().__init__(steel_grade, yield_strength)
        self.length = length  # mm
        self.area = area  # mm²
        self.radius_gyration = radius_gyration  # mm

    def calculate_slenderness(self):
        """Calculate slenderness ratio."""
        return self.length / self.radius_gyration

    def calculate_compression_capacity(self):
        """Calculate compression capacity (BS 5950, Clause 4.7)."""
        slenderness = self.calculate_slenderness()
        pc = get_compressive_strength(self.steel_grade, round(slenderness, 1))
        return self.area * pc / 1000  # kN

def main():
    # Example 4.10 parameters
    dead_load = 1000  # kN
    imposed_load = 1000  # kN
    column = ColumnDesign(
        steel_grade="S275",
        yield_strength=265,
        length=3500,  # mm (effective length)
        area=15000,  # mm² (305x305x150 UC)
        radius_gyration=77.5  # mm
    )
    
    total_load = (column.apply_load_factor(dead_load, "Dead Load") +
                  column.apply_load_factor(imposed_load, "Imposed Vertical Load"))
    capacity = column.calculate_compression_capacity()
    
    print(f"Example 4.10: Total Load = {total_load:.1f} kN")
    print(f"Compression Capacity = {capacity:.1f} kN")
    print(f"Check: {'OK' if capacity >= total_load else 'Not OK'}")

if __name__ == "__main__":
    main()

In [ ]:
# example_4_9.py
from steel_design import SteelDesign
from dataset import get_bending_strength

class BeamDesign(SteelDesign):
    """Class for beam design calculations."""
    def __init__(self, steel_grade, yield_strength, plastic_modulus, slenderness, d_t_ratio):
        super().__init__(steel_grade, yield_strength)
        self.Sx = plastic_modulus  # mm³
        self.slenderness = slenderness
        self.d_t_ratio = d_t_ratio

    def calculate_buckling_resistance_moment(self):
        """Calculate buckling resistance moment (BS 5950, Clause 4.3.6)."""
        pb = get_bending_strength(self.steel_grade, self.slenderness, self.d_t_ratio)
        return pb * self.Sx / 1e6  # kNm

def main():
    # Example 4.9 parameters (assumed continuation from 4.8)
    beam = BeamDesign(
        steel_grade="S275",
        yield_strength=275,
        plastic_modulus=2300e3,  # mm³
        slenderness=25.7,  # Different slenderness for variation
        d_t_ratio=16.8
    )
    
    buckling_moment = beam.calculate_buckling_resistance_moment()
    print(f"Example 4.9: Buckling Resistance Moment = {buckling_moment:.1f} kNm")

if __name__ == "__main__":
    main()

In [ ]:
# example_4_8.py
from steel_design import SteelDesign
from dataset import get_bending_strength

class BeamDesign(SteelDesign):
    """Class for beam design calculations."""
    def __init__(self, steel_grade, yield_strength, plastic_modulus, slenderness, d_t_ratio):
        super().__init__(steel_grade, yield_strength)
        self.Sx = plastic_modulus  # mm³
        self.slenderness = slenderness
        self.d_t_ratio = d_t_ratio

    def calculate_buckling_resistance_moment(self):
        """Calculate buckling resistance moment (BS 5950, Clause 4.3.6)."""
        pb = get_bending_strength(self.steel_grade, self.slenderness, self.d_t_ratio)
        return pb * self.Sx / 1e6  # kNm

def main():
    # Example 4.8 parameters
    beam = BeamDesign(
        steel_grade="S275",
        yield_strength=275,
        plastic_modulus=2300e3,  # mm³
        slenderness=90,  # From Table 4.9 context
        d_t_ratio=16.8
    )
    
    buckling_moment = beam.calculate_buckling_resistance_moment()
    print(f"Example 4.8: Buckling Resistance Moment = {buckling_moment:.1f} kNm")

if __name__ == "__main__":
    main()

In [ ]:
# example_4_7.py
from steel_design import SteelDesign
from dataset import get_bending_strength

class BeamDesign(SteelDesign):
    """Class for beam design calculations."""
    def __init__(self, steel_grade, yield_strength, span, plastic_modulus, d_t_ratio, slenderness):
        super().__init__(steel_grade, yield_strength)
        self.span = span  # mm
        self.S = plastic_modulus  # mm³
        self.d_t_ratio = d_t_ratio
        self.slenderness = slenderness

    def calculate_moment_capacity(self):
        """Calculate moment capacity with lateral torsional buckling (BS 5950, Clause 4.3)."""
        pb = get_bending_strength(self.steel_grade, self.slenderness, self.d_t_ratio)
        return pb * self.S / 1e6  # kNm

def main():
    # Example 4.7 parameters
    beam = BeamDesign(
        steel_grade="S275",
        yield_strength=265,
        span=7000,  # mm (assumed)
        plastic_modulus=2300e3,  # mm³ (assumed for UB)
        d_t_ratio=16.8,  # From Example 4.11 context
        slenderness=90  # From Example 4.11 context
    )
    
    moment_capacity = beam.calculate_moment_capacity()
    print(f"Example 4.7: Moment Capacity = {moment_capacity:.1f} kNm")

if __name__ == "__main__":
    main()

In [ ]:
# example_4_6.py
from steel_design import SteelDesign
from dataset import get_load_factor, get_deflection_limit

class BeamDesign(SteelDesign):
    """Class for beam design calculations."""
    def __init__(self, steel_grade, yield_strength, span, spacing, web_thickness, flange_thickness, root_radius, depth, plastic_modulus, second_moment_area):
        super().__init__(steel_grade, yield_strength)
        self.span = span  # m
        self.spacing = spacing  # m
        self.t = web_thickness  # mm
        self.T = flange_thickness  # mm
        self.r = root_radius  # mm
        self.depth = depth  # mm
        self.S = plastic_modulus  # mm³
        self.I = second_moment_area  # mm^4

    def calculate_design_load(self, load):
        """Calculate factored design load."""
        return self.apply_load_factor(load, "Imposed Vertical Load")

    def calculate_shear_force(self, load):
        """Calculate design shear force."""
        return (load * self.span) / 2  # kN

    def calculate_bending_moment(self, load):
        """Calculate design bending moment."""
        return (load * self.span ** 2) / 8  # kNm

    def calculate_moment_capacity(self):
        """Calculate moment capacity (BS 5950, Clause 4.2.5)."""
        return self.yield_strength * self.S / 1e6  # kNm

    def calculate_shear_capacity(self):
        """Calculate shear capacity (BS 5950, Clause 4.2.3)."""
        Av = self.depth * self.t  # Simplified shear area
        return 0.6 * self.yield_strength * Av / 1000  # kN

    def calculate_deflection(self, load):
        """Calculate mid-span deflection (BS 5950)."""
        return (5 * load * 1e3 * (self.span * 1000) ** 4) / (384 * self.E * self.I)  # mm

    def check_deflection(self, deflection, beam_type="Other Beams"):
        """Check deflection against Table 4.5."""
        limit_str = get_deflection_limit(beam_type)
        limit_value = (self.span * 1000) / float(limit_str.split('/')[1])
        return deflection <= limit_value

    def calculate_web_bearing_capacity(self, bearing_length):
        """Calculate web bearing capacity (BS 5950, Clause 4.5.2)."""
        k = self.T + self.r
        n = 2.5  # Assumed
        b1 = bearing_length
        return (b1 + n * k) * self.t * self.yield_strength / 1000  # kN

    def calculate_web_buckling_capacity(self, bearing_length):
        """Calculate web buckling capacity (BS 5950, Clause 4.5.3)."""
        return self.calculate_web_bearing_capacity(bearing_length)  # Simplified

def main():
    # Example 4.6 parameters (assumed due to truncation)
    beam = BeamDesign(
        steel_grade="S275",
        yield_strength=275,
        span=9,  # m
        spacing=3,  # m
        web_thickness=10,  # mm
        flange_thickness=23.7,  # mm
        root_radius=12.8,  # mm
        depth=400,  # mm
        plastic_modulus=1000e3,  # mm³ (assumed)
        second_moment_area=1e8  # mm^4 (assumed)
    )
    
    load = 1000 / (9 * 3)  # kN/m (factored load distributed over area)
    factored_load = beam.calculate_design_load(load)
    
    shear_force = beam.calculate_shear_force(factored_load)
    bending_moment = beam.calculate_bending_moment(factored_load)
    moment_capacity = beam.calculate_moment_capacity()
    shear_capacity = beam.calculate_shear_capacity()
    deflection = beam.calculate_deflection(load)
    bearing_capacity = beam.calculate_web_bearing_capacity(100)
    buckling_capacity = beam.calculate_web_buckling_capacity(100)
    
    print(f"Example 4.6: Design Load = {factored_load:.2f} kN/m")
    print(f"Shear Force = {shear_force:.2f} kN")
    print(f"Bending Moment = {bending_moment:.2f} kNm")
    print(f"Moment Capacity = {moment_capacity:.2f} kNm")
    print(f"Shear Capacity = {shear_capacity:.2f} kN")
    print(f"Deflection = {deflection:.2f} mm")
    print(f"Deflection Check: {'Pass' if beam.check_deflection(deflection) else 'Fail'}")
    print(f"Web Bearing Capacity = {bearing_capacity:.2f} kN")
    print(f"Web Buckling Capacity = {buckling_capacity:.2f} kN")

if __name__ == "__main__":
    main()

In [ ]:
# steel_design.py
from dataset import get_load_factor, get_deflection_limit, get_bending_strength, get_compressive_strength, get_effective_length_factor

class SteelDesign:
    """Base class for structural steel design to BS 5950."""
    def __init__(self, steel_grade="S275", yield_strength=275):
        self.steel_grade = steel_grade
        self.yield_strength = yield_strength  # N/mm²
        self.E = 205000  # Elastic modulus (N/mm²)

    def apply_load_factor(self, load, load_type):
        """Apply load factor from Table 4.2."""
        return load * factor

    def calculate_effective_length(self, actual_length, restraint_condition, loading_condition):
        """Calculate effective length using Table 4.6."""
        factor = get_effective_length_factor(restraint_condition, loading_condition)
        return actual_length * factor

In [ ]:
# dataset.py
import pandas as pd

# Table 4.2: Partial factors for loads (BS 5950, Table 2)
table_4_2 = {
    "Dead Load": 1.4,
    "Dead Load with Wind and Imposed Load": 1.4,
    "Imposed Vertical Load": 1.6,
}

def get_load_factor(load_type):
    """Retrieve load factor from Table 4.2."""
    return table_4_2.get(load_type, 1.4)

# Table 4.5: Suggested vertical deflection limits on beams (BS 5950, Table 8)
table_4_5 = {
    "Cantilevers": "Length/180",
    "Beams with Plaster or Brittle Finish": "Span/360",
    "Other Beams": "Span/200"
}

def get_deflection_limit(beam_type):
    """Retrieve deflection limit from Table 4.5."""
    return table_4_5.get(beam_type, "Span/200")

# Table 4.6: Effective length (Le) for beams (BS 5950, Table 13)
# Simplified due to truncation
table_4_6 = {
    "Compression Flange Fully Restrained": {
        "Normal": 1.0,  # Le/L
        "Destabilizing": 1.2
    },
    "Compression Flange Laterally Restrained": {
        "Normal": 0.85,
        "Destabilizing": 1.0
    }
}

def get_effective_length_factor(restraint_condition, loading_condition):
    """Retrieve effective length factor from Table 4.6."""
    return table_4_6.get(restraint_condition, {}).get(loading_condition, 1.0)

# Table 4.7: Bending strength (pb) based on slenderness and D/T ratio
# Simplified due to truncation
table_4_7 = {
    "S275": {
        (90, 16.8): 196,  # (slenderness λ, D/T) -> pb (N/mm²)
        (25.7, 16.8): 265,
    },
    "S355": {
        # Placeholder
    }
}

def get_bending_strength(steel_grade, slenderness, d_t_ratio):
    """Retrieve bending strength (pb) from Table 4.7."""
    return table_4_7.get(steel_grade, {}).get((slenderness, d_t_ratio), 275)

# Table 4.14: Compressive strength (pc) for strut curve c (BS 5950, Table 24(c))
table_4_14 = {
    "S275": {
        45: 222,  # slenderness (λ) -> pc (N/mm²)
        25.7: 257,
        41.2: 257,
    },
    "S355": {
        # Placeholder
    }
}

def get_compressive_strength(steel_grade, slenderness):
    """Retrieve compressive strength (pc) from Table 4.14."""
    return table_4_14.get(steel_grade, {}).get(slenderness, 275)